IMPORTS


In [ ]:
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
"""
Run inference on images, videos, directories, streams, etc.

Usage:
    $ python path/to/detect.py --source path/to/img.jpg --weights yolov5s.pt --img 640
"""

import argparse
import os
import sys
from pathlib import Path

import cv2
import numpy as np
import torch
import torch.backends.cudnn as cudnn

FILE = Path(__file__).resolve()
ROOT = FILE.parents[0]  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.experimental import attempt_load
from utils.datasets import LoadImages, LoadStreams
from utils.general import apply_classifier, check_img_size, check_imshow, check_requirements, check_suffix, colorstr, \
    increment_path, non_max_suppression, print_args, save_one_box, scale_coords, set_logging, \
    strip_optimizer, xyxy2xywh
from utils.plots import Annotator, colors
from utils.torch_utils import load_classifier, select_device, time_sync

In [ ]:
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
"""
Run inference on images, videos, directories, streams, etc.

Usage:
    $ python path/to/detect.py --source path/to/img.jpg --weights yolov5s.pt --img 640
"""

import argparse
import os
import sys
from pathlib import Path

import cv2
import numpy as np
import torch
import torch.backends.cudnn as cudnn

FILE = Path(__file__).resolve()
ROOT = FILE.parents[0]  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.experimental import attempt_load
from utils.datasets import LoadImages, LoadStreams
from utils.general import apply_classifier, check_img_size, check_imshow, check_requirements, check_suffix, colorstr, \
    increment_path, non_max_suppression, print_args, save_one_box, scale_coords, set_logging, \
    strip_optimizer, xyxy2xywh
from utils.plots import Annotator, colors
from utils.torch_utils import load_classifier, select_device, time_sync


@torch.no_grad()
def run(weights=ROOT / 'yolov5s.pt',  # model.pt path(s)
        source=ROOT / 'data/images',  # file/dir/URL/glob, 0 for webcam
        imgsz=640,  # inference size (pixels)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project=ROOT / 'runs/detect',  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        dnn=False,  # use OpenCV DNN for ONNX inference
        ):
    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
        ('rtsp://', 'rtmp://', 'http://', 'https://'))

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Initialize
    set_logging()
    device = select_device(device)
    half &= device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    w = str(weights[0] if isinstance(weights, list) else weights)
    classify, suffix, suffixes = False, Path(w).suffix.lower(), ['.pt', '.onnx', '.tflite', '.pb', '']
    check_suffix(w, suffixes)  # check weights have acceptable suffix
    pt, onnx, tflite, pb, saved_model = (suffix == x for x in suffixes)  # backend booleans
    stride, names = 64, [f'class{i}' for i in range(1000)]  # assign defaults
    if pt:
        model = torch.jit.load(w) if 'torchscript' in w else attempt_load(weights, map_location=device)
        stride = int(model.stride.max())  # model stride
        names = model.module.names if hasattr(model, 'module') else model.names  # get class names
        if half:
            model.half()  # to FP16
        if classify:  # second-stage classifier
            modelc = load_classifier(name='resnet50', n=2)  # initialize
            modelc.load_state_dict(torch.load('resnet50.pt', map_location=device)['model']).to(device).eval()
    elif onnx:
        if dnn:
            # check_requirements(('opencv-python>=4.5.4',))
            net = cv2.dnn.readNetFromONNX(w)
        else:
            check_requirements(('onnx', 'onnxruntime-gpu' if torch.has_cuda else 'onnxruntime'))
            import onnxruntime
            session = onnxruntime.InferenceSession(w, None)
    else:  # TensorFlow models
        check_requirements(('tensorflow>=2.4.1',))
        import tensorflow as tf
        if pb:  # https://www.tensorflow.org/guide/migrate#a_graphpb_or_graphpbtxt
            def wrap_frozen_graph(gd, inputs, outputs):
                x = tf.compat.v1.wrap_function(lambda: tf.compat.v1.import_graph_def(gd, name=""), [])  # wrapped import
                return x.prune(tf.nest.map_structure(x.graph.as_graph_element, inputs),
                               tf.nest.map_structure(x.graph.as_graph_element, outputs))

            graph_def = tf.Graph().as_graph_def()
            graph_def.ParseFromString(open(w, 'rb').read())
            frozen_func = wrap_frozen_graph(gd=graph_def, inputs="x:0", outputs="Identity:0")
        elif saved_model:
            model = tf.keras.models.load_model(w)
        elif tflite:
            interpreter = tf.lite.Interpreter(model_path=w)  # load TFLite model
            interpreter.allocate_tensors()  # allocate
            input_details = interpreter.get_input_details()  # inputs
            output_details = interpreter.get_output_details()  # outputs
            int8 = input_details[0]['dtype'] == np.uint8  # is TFLite quantized uint8 model
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Dataloader
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt)
        bs = len(dataset)  # batch_size
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
        bs = 1  # batch_size
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    if pt and device.type != 'cpu':
        model(torch.zeros(1, 3, *imgsz).to(device).type_as(next(model.parameters())))  # run once
    dt, seen = [0.0, 0.0, 0.0], 0
    for path, img, im0s, vid_cap in dataset:
        t1 = time_sync()
        if onnx:
            img = img.astype('float32')
        else:
            img = torch.from_numpy(img).to(device)
            img = img.half() if half else img.float()  # uint8 to fp16/32
        img = img / 255.0  # 0 - 255 to 0.0 - 1.0
        if len(img.shape) == 3:
            img = img[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        if pt:
            visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
            pred = model(img, augment=augment, visualize=visualize)[0]
        elif onnx:
            if dnn:
                net.setInput(img)
                pred = torch.tensor(net.forward())
            else:
                pred = torch.tensor(session.run([session.get_outputs()[0].name], {session.get_inputs()[0].name: img}))
        else:  # tensorflow model (tflite, pb, saved_model)
            imn = img.permute(0, 2, 3, 1).cpu().numpy()  # image in numpy
            if pb:
                pred = frozen_func(x=tf.constant(imn)).numpy()
            elif saved_model:
                pred = model(imn, training=False).numpy()
            elif tflite:
                if int8:
                    scale, zero_point = input_details[0]['quantization']
                    imn = (imn / scale + zero_point).astype(np.uint8)  # de-scale
                interpreter.set_tensor(input_details[0]['index'], imn)
                interpreter.invoke()
                pred = interpreter.get_tensor(output_details[0]['index'])
                if int8:
                    scale, zero_point = output_details[0]['quantization']
                    pred = (pred.astype(np.float32) - zero_point) * scale  # re-scale
            pred[..., 0] *= imgsz[1]  # x
            pred[..., 1] *= imgsz[0]  # y
            pred[..., 2] *= imgsz[1]  # w
            pred[..., 3] *= imgsz[0]  # h
            pred = torch.tensor(pred)
        t3 = time_sync()
        dt[1] += t3 - t2

        # NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # Second-stage classifier (optional)
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process predictions
        for i, det in enumerate(pred):  # per image
            seen += 1
            if webcam:  # batch_size >= 1
                p, s, im0, frame = path[i], f'{i}: ', im0s[i].copy(), dataset.count
            else:
                p, s, im0, frame = path, '', im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        annotator.box_label(xyxy, label, color=colors(c, True))
                        if save_crop:
                            save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

            # Print time (inference-only)
            print(f'{s}Done. ({t3 - t2:.3f}s)')

            # Stream results
            im0 = annotator.result()
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path[i] != save_path:  # new video
                        vid_path[i] = save_path
                        if isinstance(vid_writer[i], cv2.VideoWriter):
                            vid_writer[i].release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                            save_path += '.mp4'
                        vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer[i].write(im0)

    # Print results
    t = tuple(x / seen * 1E3 for x in dt)  # speeds per image
    print(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}' % t)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        print(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(weights)  # update model (to fix SourceChangeWarning)


def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', nargs='+', type=str, default=ROOT / 'yolov5s.pt', help='model path(s)')
    parser.add_argument('--source', type=str, default=ROOT / 'data/images', help='file/dir/URL/glob, 0 for webcam')
    parser.add_argument('--imgsz', '--img', '--img-size', nargs='+', type=int, default=[640], help='inference size h,w')
    parser.add_argument('--conf-thres', type=float, default=0.25, help='confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.45, help='NMS IoU threshold')
    parser.add_argument('--max-det', type=int, default=1000, help='maximum detections per image')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--view-img', action='store_true', help='show results')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--save-crop', action='store_true', help='save cropped prediction boxes')
    parser.add_argument('--nosave', action='store_true', help='do not save images/videos')
    parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --classes 0, or --classes 0 2 3')
    parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--visualize', action='store_true', help='visualize features')
    parser.add_argument('--update', action='store_true', help='update all models')
    parser.add_argument('--project', default=ROOT / 'runs/detect', help='save results to project/name')
    parser.add_argument('--name', default='exp', help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--line-thickness', default=3, type=int, help='bounding box thickness (pixels)')
    parser.add_argument('--hide-labels', default=False, action='store_true', help='hide labels')
    parser.add_argument('--hide-conf', default=False, action='store_true', help='hide confidences')
    parser.add_argument('--half', action='store_true', help='use FP16 half-precision inference')
    parser.add_argument('--dnn', action='store_true', help='use OpenCV DNN for ONNX inference')
    opt = parser.parse_args()
    opt.imgsz *= 2 if len(opt.imgsz) == 1 else 1  # expand
    print_args(FILE.stem, opt)
    return opt


def main(opt):
    check_requirements(exclude=('tensorboard', 'thop'))
    run(**vars(opt))


if __name__ == "__main__":
    opt = parse_opt()
    main(opt)


# Montar Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [ ]:
!pip install cvzone
!pip install mediapipe

  Created wheel for cvzone: filename=cvzone-1.5.2-py3-none-any.whl size=18356 sha256=562d8862db53f7c22ee79f55de890d3e472d6e2f5b5bb23bf046ab67aed8aa62
  Stored in directory: /root/.cache/pip/wheels/5b/97/a1/601bf3f23368b10a67550cb6154cec1717eb03e5ea17ebdb7b
Successfully built cvzone
     |████████████████████████████████| 34.0 MB 51 kB/s 


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import codecs
import cvzone
import os
from cvzone.SelfiSegmentationModule import SelfiSegmentation
from pathlib import Path
import sys
from google.colab.patches import cv2_imshow
path = '/content/drive/MyDrive/DPVSA/YOLOv5/ColorDetection/'

# Class Dict


In [ ]:
clasDict = {
  "referee": 0,
  "player_team_2": 1,
  "goalkeeper_team_2": 2,
  "outsider": 3,
  "player_team_1": 4,
  "goalkeeper_team_1": 5,
  "ball": 6
}

# Color Detection

In [ ]:
def cut_box(image, coordinates, cropCoordinates):

  col1 = coordinates[0]
  row1 = coordinates[1]
  col2 = coordinates[2]
  row2 = coordinates[3]
  box = image[row1:row2,col1:col2]

  col1 = cropCoordinates[0]
  row1 = cropCoordinates[1]
  col2 = cropCoordinates[2]
  row2 = cropCoordinates[3]
  cropBox = image[row1:row2,col1:col2]

  #cv2_imshow(box)
  #cv2_imshow(cropBox)

  return box, cropBox

In [ ]:
def color_percentage(colorPixels,coordinates = 0, totalPixels = 0, mode = 0):

  if mode == 0: # no background mode
    percentage = colorPixels/totalPixels
  else: # background mode
    width = coordinates[0] - coordinates[2]
    height = coordinates[1] - coordinates[3]
    percentage = colorPixels/(width*height)

  return percentage

In [ ]:
def color_detection(image, coordinates, cropped = False):

  # TODO REVISAR CORES
  hsvImage = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  colorCount = {
      'red':0,
      'blue':0,
      'yellow':0,
      'green':0,
      'white':0,
      'black':0
  }


  # BACKGROUND SEGMENTATION MASK
  pureBlackRange = np.array([[0,0,0],[0,0,0]])
  pureBlackMask = cv2.inRange(hsvImage,pureBlackRange[0],pureBlackRange[1])
  pureBlackMask = np.where(pureBlackMask == 255, 1, pureBlackMask)
  pureBlackMask = np.where(pureBlackMask == 0, 255, pureBlackMask) 
  pureBlackMask = np.where(pureBlackMask == 1, 0, pureBlackMask)
  notBackground = np.count_nonzero(pureBlackMask == 255)

  if(cropped):
    notBackground = image.shape[0]*image.shape[1]
    

  #print('PURE BLACK MASK')   
  #cv2_imshow(pureBlackMask)

  # RED MASK  
  redRange1 = np.array([[0,70,50],[20,255,255]])
  redRange2 = np.array([[170,70,50],[180,255,255]])
  maskRed1 = cv2.inRange(hsvImage,redRange1[0],redRange1[1])
  maskRed2= cv2.inRange(hsvImage,redRange2[0],redRange2[1])
  maskRed3 = cv2.bitwise_or(maskRed1, maskRed2)
  maskRed4 = cv2.bitwise_and(maskRed3, pureBlackMask)
  redOcurrence = np.count_nonzero(maskRed4 == 255)
  redPerc = color_percentage(redOcurrence, totalPixels = notBackground, mode = 0 )
  colorCount['red'] = redPerc
  #print('redPerc',redPerc)

  # WHITE MASK
  whiteRange = np.array([[0,0,170],[179,38,255]])
  maskWhite1 = cv2.inRange(hsvImage,whiteRange[0],whiteRange[1])
  maskWhite2 = cv2.bitwise_and(maskWhite1, pureBlackMask)
  whiteOcurrence = np.count_nonzero(maskWhite2 == 255)
  whitePerc = color_percentage(whiteOcurrence, totalPixels = notBackground, mode = 0 )
  colorCount['white'] = whitePerc
  #print('whitePerc',whitePerc)

  # GREEN MASK
  greenRange = np.array([[37,66,102],[81,255,255]])
  maskGreen1 = cv2.inRange(hsvImage,greenRange[0],greenRange[1])
  maskGreen2 = cv2.bitwise_and(maskGreen1, pureBlackMask)
  greenOcurrence = np.count_nonzero(maskGreen2 == 255)
  greenPerc =  color_percentage(greenOcurrence, totalPixels = notBackground, mode = 0)
  colorCount['green'] = greenPerc
  #print('greenPerc',greenPerc)

  #BLUE MASK
  blueRange = np.array([[85,70,50],[140,255,255]])
  maskBlue1 = cv2.inRange(hsvImage,blueRange[0],blueRange[1])
  maskBlue2 =  cv2.bitwise_and(maskBlue1, pureBlackMask)
  blueOcurrence = np.count_nonzero(maskBlue2 == 255)
  bluePerc = color_percentage(blueOcurrence, totalPixels = notBackground, mode = 0)
  colorCount['blue'] = bluePerc
  #print('BluePerc',bluePerc)


  # YELLOW MASK
  yellowRange = np.array([[23,38,102],[33,255,255]])
  maskYellow1 = cv2.inRange(hsvImage,yellowRange[0],yellowRange[1])
  maskYellow2 = cv2.bitwise_and(maskYellow1, pureBlackMask)
  yellowOcurrence = np.count_nonzero(maskYellow2 == 255)
  yellowPerc  = color_percentage(yellowOcurrence, totalPixels = notBackground, mode = 0)
  colorCount['yellow'] = yellowPerc
  #print('yellowPerc', yellowPerc)

  # BLACK MASK
  blackRange = np.array([[0,0,0],[255,70,50]])
  maskBlack1 = cv2.inRange(hsvImage,blackRange[0],blackRange[1])
  maskBlack2 = cv2.bitwise_and(maskBlack1, pureBlackMask)
  blackOcurrence = np.count_nonzero(maskBlack2 == 255)
  blackPerc = color_percentage(blackOcurrence, totalPixels = notBackground, mode = 0)
  colorCount['black'] = blackPerc
  #print('blackPerc',blackPerc)

  color = max(colorCount,key=colorCount.get) 
  if(color == 'green'):
    colorCount.pop(color, None)
    secondColor = max(colorCount,key=colorCount.get)
    if colorCount[secondColor] > 0.1:
      color = secondColor
    else:
      colorCount['green'] = greenPerc
      color = max(colorCount,key=colorCount.get) 


  colorPercentage = colorCount[color]
  return color,colorPercentage

In [ ]:
def color_recognition(orImage, segImage, cropImage, coordinates, cropCoordinates):

  backgroundPixels = np.count_nonzero(segImage == 0) / 3
  backgroundPerc = backgroundPixels / (orImage.shape[0] * orImage.shape[1])
  #print(backgroundPerc)

  if(backgroundPerc > 0.9):
    #print('PRETEOU')
    color, percentage = color_detection(cropImage, cropCoordinates, cropped=True)
  else:
    #print('NAO PRETEOU')
    color, percentage = color_detection(segImage, coordinates)

  return color, percentage

In [ ]:
def pixel_position(boxplot, imWidth, imHeight):

  boxClass = boxplot[0]
  xCenter = boxplot[1]
  yCenter = boxplot[2]
  width = boxplot[3]
  height = boxplot[4]

  xTopLeft = int((xCenter - width*0.5) * imWidth) 
  yTopLeft = int((yCenter - height*0.5) * imHeight) 
  xBottomRight = int((xCenter + (width*0.5))*imWidth)
  yBottomRight = int((yCenter + (height*0.5))*imHeight) 

  coordinates = [xTopLeft, yTopLeft, xBottomRight, yBottomRight]

  # Cropped Coordinates

  croppedWidth = boxplot[3]*0.5
  croppedHeight = boxplot[4]*0.7
  
  cropXTopLeft = int((xCenter - croppedWidth*0.5) * imWidth) 
  cropYTopLeft = int((yCenter - croppedHeight*0.5) * imHeight) 
  cropXBottomRight = int((xCenter + (width*0.5))*imWidth)
  CropYBottomRight = int((yCenter + (height*0.5))*imHeight) -10

  cropCoordinates = [cropXTopLeft, cropYTopLeft, cropXBottomRight, CropYBottomRight]

  return boxClass, coordinates, cropCoordinates

In [ ]:
def color_inference(image,labels):
  labels = np.loadtxt(path+'21062018_55410.txt')
  image = cv2.imread(path+'21062018_55410.png')
  cv2_imshow(image)
  segmentor = SelfiSegmentation()
  for row in labels:
    if row[0] == 4 :
      boxClass, coordinates, cropCoordinates = pixel_position(row,image.shape[1], image.shape[0])
      box, cropBox = cut_box(image, coordinates, cropCoordinates)
      segmentedBox = segmentor.removeBG(box,(0,0,0),threshold=0.15)
      #segmentedBox2 = segmentor.removeBG(box,(0,0,0),threshold=0.5)
      color, percentage = color_recognition(box, segmentedBox, cropBox, coordinates, cropCoordinates)
      print(color, percentage)
      cv2_imshow(box)
      #cv2_imshow(segmentedBox)
      #cv2_imshow(segmentedBox2)
      cv2_imshow(cropBox)
      print("\n")

In [ ]:
def detection(image, labels, show_crop=False):
  camisas = {}
  posicao_camisa = {}
  segmentor = SelfiSegmentation()
  i=0
  for row in labels:
    if(row[0]==4) or (row[0]==1):
      norm_cordinates = [row[1],row[2],row[3],row[4]]
      boxClass, coordinates, cropCoordinates = pixel_position(row, image.shape[1], image.shape[0])
      box, cropBox = cut_box(image, coordinates, cropCoordinates)
      segmentedBox = segmentor.removeBG(box,(0,0,0),threshold=0.1)
      finalColor, percentage = color_recognition(box, segmentedBox, cropBox, coordinates, cropCoordinates)
      posicao_camisa["linha {}".format(i)] = [norm_cordinates,coordinates,finalColor]
      i+=1
      #Lógica de contagem das cores das camisas
      if finalColor not in camisas.keys():
        camisas[finalColor] = 1
      else:
        camisas[finalColor] += 1
      
      #Mostra as informações para debug
      if show_crop:
        print(percentage)
        print(finalColor)
        cv2_imshow(box)
        #cv2_imshow(segmentedBox)
        #cv2.rectangle(image, (xTopLeft, yTopLeft), (xBottomRight, yBottomRight), (255, 0, 0), 2)
        #row[0] = 666
  return camisas, posicao_camisa
#cores = detection(image,labels)
#cv2_imshow(image)
#[cor,pt1,pt2]

In [160]:
def altera_labels(labels_file, labels_file_outuput, img, camisas_cores, posicao_bbox):
  escolha_time = camisas_cores.copy()
  color_time_1_c4 = max(escolha_time, key=escolha_time.get)
  #{'yellow': 10, 'white': 5, 'red': 3, 'green': 2}
  escolha_time.pop(color_time_1_c4, None)
  color_time_2_c1 = max(escolha_time, key=escolha_time.get)
  
  teste = labels_file
  #Escrita das classes e conversão dos valores
  with open(labels_file, "r") as f:
    with open(labels_file_outuput, "w") as d:
      i = 0
      for line in f:
        if (line.startswith("4")) or (line.startswith("5")):
          num_linha = "linha {}".format(i)
          #a = str(line.split()[1])
          #b = str(line.split()[2])
          #d.write("{} {}\n".format(posicao_bbox[num_linha][0][0] == float(a), posicao_bbox[num_linha][0][1] == float(b)))
          conf = line.split()[-1]
          if posicao_bbox[num_linha][0][0] == float(line.split()[1]) and posicao_bbox[num_linha][0][1] == float(line.split()[2]):
            if posicao_bbox[num_linha][2] == color_time_1_c4:
              nova_classe = 4
              d.write("{} {} {} {} {} {}\n".format(posicao_bbox[num_linha][1][0],posicao_bbox[num_linha][1][1],posicao_bbox[num_linha][1][2],posicao_bbox[num_linha][1][3],conf,nova_classe))
            elif posicao_bbox[num_linha][2] == color_time_2_c1:
              nova_classe = 1
              d.write("{} {} {} {} {} {}\n".format(posicao_bbox[num_linha][1][0],posicao_bbox[num_linha][1][1],posicao_bbox[num_linha][1][2],posicao_bbox[num_linha][1][3],conf,nova_classe))
            else: #Caso a cor detectada seja diferente das dos dois times, decidimos por quantidade de jogadores 
              if camisas_cores[color_time_1_c4]>=camisas_cores[color_time_2_c1]:
                nova_classe = 1
                d.write("{} {} {} {} {} {}\n".format(posicao_bbox[num_linha][1][0],posicao_bbox[num_linha][1][1],posicao_bbox[num_linha][1][2],posicao_bbox[num_linha][1][3],conf,nova_classe))
              if camisas_cores[color_time_1_c4]<camisas_cores[color_time_2_c1]:
                nova_classe = 4
                d.write("{} {} {} {} {} {}\n".format(posicao_bbox[num_linha][1][0], posicao_bbox[num_linha][1][1],posicao_bbox[num_linha][1][2],posicao_bbox[num_linha][1][3],conf,nova_classe))
          i += 1
        else:
          image = cv2.imread(img)
          boxplot = line.split()
          
          conf = float(boxplot[-1])
          boxClass = boxplot[0]
          xCenter = float(boxplot[1])
          yCenter = float(boxplot[2])
          width = float(boxplot[3])
          height = float(boxplot[4])

          xTopLeft = int((xCenter - width*0.5) *  image.shape[1]) 
          yTopLeft = int((yCenter - height*0.5) * image.shape[0]) 
          xBottomRight = int((xCenter + (width*0.5))* image.shape[1])
          yBottomRight = int((yCenter + (height*0.5))*image.shape[0])
          
          #boxClass, new_coordinates,_ = pixel_position(label, image.shape[1], image.shape[0])
          d.write("{} {} {} {} {} {}\n".format(xTopLeft, yTopLeft,xBottomRight,  yBottomRight, conf, boxClass))

def count_files(folder):
  print("folder has " + str(len(os.listdir(folder))) + " files")

In [161]:
img_list=[img for img in sorted(Path('/content/drive/MyDrive/DPVSA/YOLOv5/ColorDetection/teste/images_video').glob('*.png'))]
label_list=[label for label in sorted(Path('/content/drive/MyDrive/DPVSA/YOLOv5/ColorDetection/teste/labels_video').glob('*.txt'))]
label_converted_list=[label for label in sorted(Path('/content/drive/MyDrive/DPVSA/YOLOv5/ColorDetection/teste/labels_convertidas').glob('*.txt'))] #os arquivos serão alterados aqui

In [ ]:
"""def define_camisas(todas_camisas_video):
  camisas = todas_camisas_video
  camisa_time1 = max(camisas, key=camisas.get)
  camisas.pop(camisa_time1, None) 
  camisa_time2 = max(camisas, key=camisas.get)
  return camisa_time1, camisa_time2"""

In [ ]:
"""cores_totais = {}
for img,label in zip(img_list,label_list): #loop detecção de cor
  labels2read = np.loadtxt(str(label))
  image = cv2.imread(str(img))
  cores,posicao_camisa = detection(image,labels2read,show_crop=True)
  cv2_imshow(image)
  print("{}: {}".format(str(img)[57:],cores))
  for cor in cores.keys():
    if cor in cores_totais.keys():
      cores_totais[cor] += cores[cor]
    else: 
      cores_totais[cor] = cores[cor]

cor_time1, cor_time2 = define_camisas(cores_totais)
for img,label,converted_label in zip(img_list,label_list,label_converted_list): #loop alteração de classes
  label_path = str(label)
  converted_label_path = str(converted_label)
  img_path = str(img)
  altera_labels(label_path, converted_label_path, img_path, cor_time1, cor_time2, posicao_camisa)"""

In [162]:
for img,label,converted_label in zip(img_list,label_list,label_converted_list):
  labels = np.loadtxt(str(label))
  image = cv2.imread(str(img))
  cores,posicao_camisa = detection(image,labels)#,show_crop=True)
  #cv2_imshow(image)
  print("{}: {}".format(str(img)[57:],cores))
  label_path = str(label)
  converted_label_path = str(converted_label)
  img_path = str(img)
  altera_labels(label_path, converted_label_path, img_path, cores, posicao_camisa)

In [163]:
def teste(coordinates):

  x1 = coordinates[0]
  y1 = coordinates[1]
  x2 = coordinates[2]
  y2 = coordinates[3]
  classe = coordinates[5]
  start = (x1,y1)
  finish = (x2,y2)

  #cv2_imshow(box)
  #cv2_imshow(cropBox)

  return start, finish, classe

In [164]:
for img,converted_label in zip(img_list,label_converted_list):
  image = cv2.imread(str(img))
  file = np.loadtxt(str(converted_label))
  cores = {
    0: (255, 0, 0), #"referee" --red
    1: (0,0,0), #"player_team_2" --balck
    2: (0,255,0), #"goalkeeper_team_2" --green
    3: (255,255,0), #"outsider" --yellow
    4: (255,255,255), #"player_team_1" --white
    5: (255,0,255), #"goalkeeper_team_1" --pink
    6: (0,0,255)} #"ball" --blue
  for row in file:
    x1 = int(row[0])
    y1 = int(row[1])
    x2 = int(row[2])
    y2 = int(row[3])
    classe = int(row[5])
    color = cores[classe]
    #print(type(x1))
    cv2.rectangle(image, (x1,y1), (x2,y2), color)
  cv2_imshow(image) 

In [150]:
cap = cv2.VideoCapture("/content/drive/MyDrive/DPVSA/video_teste.mp4")
i = 0
#os.chdir("/content/drive/MyDrive/DPVSA/YOLOv5/ColorDetection/teste/images_video")
while(cap.isOpened()):
    ret, frame = cap.read()
     
    # This condition prevents from infinite looping
    # incase video ends.
    if i == 10:
      break
    if ret == False:
      break
     
    # Save Frame by Frame into disk using imwrite method
    cv2.imwrite('/content/drive/MyDrive/DPVSA/YOLOv5/ColorDetection/teste/images_video/Cópia de video_teste_{}.jpg'.format(i+1), frame)
  
    i += 1
 
cap.release()
cv2.destroyAllWindows()

## OUTPUT

The output should be a list of bounding box arrays with the format (x1, y1, x2, y2, conf, class) for each frame.

x1, y1: Top left

x2, y2: Bottom Right 

In [ ]:
result_frame_N = [(397.5460122699385, 771.5521472392636, 458.89570552147234, 938.4233128834355, 0.8123, 3.0), ...]